In [1]:
import pandas as pd
import gc
import pickle

In [ ]:
from collections import defaultdict,Counter
import itertools
import difflib

In [2]:
from tqdm import tqdm
tqdm.pandas()  # 注册 pandas 的 progress_apply

In [4]:
df=pd.read_csv('data/invest_patent_all_3_merge_vc.csv',index_col=0)

根据这里的企业名称和交易时间，结合专利数据库，再构建几个新的变量：在每一个企业交易时间之后的1-5年时间里（分别1、2、3、4、5年进行统计），该企业的各类专利数量的总和

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21729 entries, 0 to 38512
Data columns (total 94 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   投资方名称_投资方                    21729 non-null  object 
 1   机构方名称全称_投资方                  21729 non-null  object 
 2   行政区划代码_投资方                   21729 non-null  int64  
 3   省份_投资方                       21729 non-null  object 
 4   城市_投资方                       21729 non-null  object 
 5   区县_投资方                       21729 non-null  object 
 6   统一社会信用代码_投资方                 21729 non-null  object 
 7   经度高德_投资方                     21489 non-null  float64
 8   纬度高德_投资方                     21489 non-null  float64
 9   成立年份_投资方                     21729 non-null  float64
 10  社保人数_投资方                     17886 non-null  float64
 11  从业人数_投资方                     16715 non-null  float64
 12  社保人数17版_投资方                  21489 non-null  float64
 13  从业人数17版_投资方          

In [8]:
df.交易时间=pd.to_datetime(df.交易时间)

In [9]:
df.企业全称.nunique()

13579

In [18]:
df_pantent=pd.read_csv('data/source/patent_all_2024_first.tsv',sep='\t')#'data/source/patent_all_2411.csv
df_pantent.shape

(22255063, 17)

In [15]:
df_pantent.columns

Index(['标题 (中文)', '申请人', '公开（公告）号', '公开（公告）日', '申请号', '申请日', '专利类型', 'IPC主分类',
       '当前权利人', '工商统一社会信用代码', '工商上市代码', '引证专利', '引证申请人', '被引证申请人', '当前法律状态',
       '首次公开日', 'table_name'],
      dtype='object')

In [16]:
df_pantent.table_name.value_counts()

table_name
patent_incopat_2016b_202411     6688311
patent_incopat_202101_202211    5586621
patent_incopat_2020b_202411     5464605
patent202210                    1576086
patent202404                    1492032
patent202410                     688260
patent_incopat_202212_202312     586203
patent202309                      93815
patent202408                      46033
patent202407                      33097
Name: count, dtype: int64

In [21]:
%%time
companies=set(df.企业全称.dropna().unique())
df_pantent['企业全称'] = df_pantent['申请人'].str.split(';').apply(lambda x: [i.strip() for i in x if i.strip() in companies] if isinstance(x, list) else [])
mask = (df_pantent['企业全称'].map(len)>0)
mask.sum()

CPU times: total: 25.6 s
Wall time: 26.2 s


np.int64(1234629)

In [23]:
patent_filter=df_pantent[mask].copy()
patent_filter.申请日=pd.to_datetime(patent_filter.申请日)
patent_filter['公开（公告）日']=pd.to_datetime(patent_filter['公开（公告）日'])
patent_filter = patent_filter.sort_values(['申请日','公开（公告）日'])
patent_filter['企业全称'].explode().nunique()#.str.strip(';')

9879

In [42]:
#patent_filter['企业全称']=patent_filter['企业全称'].map(';'.join)

In [50]:
patent_filter.groupby(['table_name', '专利类型']).size()

table_name                    专利类型
patent202210                  发明授权     95582
                              发明申请    111840
                              外观设计     38082
                              实用新型    158937
patent202309                  发明授权       443
                              发明申请       571
                              外观设计        32
                              实用新型        43
patent202404                  发明授权     76189
                              发明申请    217605
                              外观设计     26458
                              实用新型    136173
patent202407                  发明授权        89
                              发明申请       163
                              外观设计        66
                              实用新型        84
patent202408                  发明授权       364
                              发明申请      1026
                              外观设计       245
                              实用新型       698
patent202410                  发明授权      1761
                    

In [24]:
len(companies)

13579

In [26]:
patent_co=patent_filter.explode('企业全称', ignore_index=True)#.reset_index()
patent_co.shape

(1237447, 18)

In [37]:
all_results = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    ent, trade_date = row['企业全称'], row['交易时间']
    ent_patents = patent_co[patent_co['企业全称'] == ent]
    data = {
        'idx':_,
        '企业全称': ent,
        '交易时间': trade_date
    }
    for year in range(1, 6):
        end_date = trade_date + pd.DateOffset(years=year)
        #delta = int((grant_date - trade_date).days / 365.25)
        filtered = ent_patents[(ent_patents['申请日'] > trade_date) & (ent_patents['申请日'] <= end_date)]
        if filtered.empty:
            continue
        counts = filtered.groupby('专利类型')['申请号'].count().to_dict()
        for ptype, count in counts.items():
            col = f'{ptype}_专利数_交易后_{year}年'
            data[col] = count

    all_results.append(data)

    # df.loc[_,data.keys]=pd.Series( data)

100%|████████████████████████████████████████████████████████████████████████████| 21729/21729 [30:18<00:00, 11.95it/s]


In [36]:
filtered.groupby('专利类型')['申请号'].count()

专利类型
发明申请    2
Name: 申请号, dtype: int64

In [38]:
result_df = pd.DataFrame(all_results).set_index('idx')
# df.update(result_df)
result_df.to_excel('data/invest_patent_all_3_merge_vc_result.xlsx')
result_df.shape

(21729, 22)

In [40]:
df = df.merge(pd.DataFrame(all_results), left_index=True, right_on='idx', how='left')# on=['企业全称', '交易时间']

In [41]:
df.to_csv('data/invest_patent_all_3_merge_vc_2505.csv')

In [ ]:
expanded = pd.concat([
    df.assign(
        years=n,
        start_date=df['交易时间'],
        end_date=df['交易时间'] + pd.DateOffset(years=n)
    )
    for n in range(1, 6)
])

results = []
for _, row in expanded.iterrows():
    subset = patent_co[
        (patent_co['企业全称'] == row['企业全称']) &
        (patent_co['申请日'] > row['start_date']) &
        (patent_co['申请日'] <= row['end_date'])
    ]
    counts = subset['专利类型'].value_counts().to_dict()
    record = {'企业全称': row['企业全称'],'交易时间': row['交易时间'],'years': row['years']}
    for ptype, count in counts.items():
        record[f'{ptype}_{row["years"]}y'] = count
    results.append(record)


result_df = pd.DataFrame(results)
final = result_df.groupby(['企业全称', '交易时间']).first().reset_index()
# final = final.fillna(0)
# for col in final.columns:
#     if '_y' in col:
#         final[col] = final[col].astype(int)

匹配一下 我们之前做初创公司的这些专利信息并导出

In [2]:
df=pd.read_excel('data\\invest_all_2506.xlsx')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155309 entries, 0 to 155308
Data columns (total 57 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  155309 non-null  int64  
 1   企业简称        155216 non-null  object 
 2   企业全称        155309 non-null  object 
 3   业务描述        124343 non-null  object 
 4   所在行业        155306 non-null  object 
 5   所在地         151979 non-null  object 
 6   企业简介        155214 non-null  object 
 7   详细地址        131135 non-null  object 
 8   联系电话        127775 non-null  object 
 9   企业网址        69271 non-null   object 
 10  邮箱          116135 non-null  object 
 11  成立时间        142664 non-null  object 
 12  统一信用代码      139372 non-null  object 
 13  法人代表        130608 non-null  object 
 14  注册资本        130603 non-null  object 
 15  经营状态        130609 non-null  object 
 16  经营范围        130408 non-null  object 
 17  注册地址        139709 non-null  object 
 18  上市状态        155309 non-null  object 
 19  股票

In [6]:
df.交易时间.max()

'2024-05-21'

In [7]:
df.shape

(155309, 57)

In [8]:
df.head()

,Unnamed: 0,企业简称,企业全称,业务描述,所在行业,所在地,企业简介,详细地址,联系电话,企业网址,...,投资方交易所,退出方名称,退出方基金名称,退出机构类型,退出方式,退出方上市状态,退出方股票代码,退出方交易所,中介名称,中介类型
0,0,小牛顿教育,北京小牛顿科学启蒙教育科技有限公司,儿童科学启蒙教育平台,教育培训-K12,北京市-海淀区,北京小牛顿科学启蒙教育科技有限公司，成立于2003年，是一家专注于幼少儿科学启蒙教育的企业，...,北京市海淀区中关村大街甲59号文化大厦16层1606室,010-64452203,http://www.xnd.cn,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,极智嘉,北京极智嘉科技股份有限公司,物流机器人研发商,先进制造-机器人,北京市-朝阳区,北京极智嘉科技股份有限公司（曾用名：北京极智嘉科技有限公司）成立于2015年，旗下Geek+...,北京市朝阳区创远路36号院1号楼1层101室 \r\n北京市朝阳区北苑路30号北京文化创意大...,010-84824981 \r\n +86 4000 450 010,http://www.geekplus.com.cn,...,\r\n \r\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,你好植物,成都你好植物科技有限公司,NaN,消费-餐饮,四川省-成都市,成都你好植物科技有限公司成立于2013年4月8日，是一家由十年以上快消品行业经验的年青团队创...,成都市高新区西区大道99号伯雅科技园,028-67707575,www.nihaozhiwu.com,...,\r\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Silot,Silot,NaN,金融-科技金融,新加坡,Silot是一家以“连接”为主要特征的创业公司。Silot定位为消费金融领域的Amedeus...,NaN,NaN,https://www.silots.com/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,华宿电气,上海华宿电气股份有限公司,电气防火产品研发商,传统制造-机械设备,上海市-浦东新区,上海华宿电气股份有限公司成立于2008年3月24日，是一家专业从事电力及电气安全产品研发、生...,上海市张江高科技园区达尔文路88号6号楼1楼 \r\n中国（上海）自由贸易试验区盛荣路88弄...,+86-21-51330568 \r\n021-50663362,www.huasu.net,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
companies=set(df.企业全称.dropna().unique())
len(companies)

83793

In [11]:
patent_all_invest=pd.read_csv('data/patent_all_invest_2024.tsv',sep='\t',nrows=5)
patent_all_invest

,序号,标题 (中文),标题 (英文),摘要 (中文),摘要 (英文),申请人,公开（公告）号,公开（公告）日,申请号,申请日,...,DWPI同族专利,DWPI同族国家/地区,DWPI同族ID,DWPI专利权人/申请人,DWPI发明人,DWPI优先权国别-最早,DWPI优先权号,DWPI优先权国别,table_name,invest_申请人
0,107,高炉中心加焦装置,Device for adding coke at center of blast furnace,本实用新型公开了一种能实现向高炉中心加焦 的新装置，通过安装在炉顶受料漏斗内的翻板可以顺 利...,The utility model discloses a device for addin...,华东冶金学院; 马鞍山钢铁股份有限公司,CN2170307Y,1994-06-29,CN93237739.4,1993-10-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,patent202210,马鞍山钢铁股份有限公司
1,119,炼钢炉坑复合轨道板,Complex track plate for steel-smelting furnace...,本实用新型涉及一种炼钢炉炉坑轨道板，用作钢 包车、渣车运行的轨道。其特征在于：采用钢板制作 ...,The utility model relates to a complex track p...,马鞍山钢铁股份有限公司,CN2204409Y,1995-08-02,CN94226705.2,1994-01-27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,patent202210,马鞍山钢铁股份有限公司
2,124,无级调整液压捣炉扎眼机,Stepless regulating hydraulic furnace-ramming ...,本实用新型涉及一种矿热炉专用机械设备。主 要由扎进翻料、转盘、行车、固定等机构以及无级调整 ...,The utility model relates to a mechanical equi...,马鞍山钢铁股份有限公司,CN2189729Y,1995-02-15,CN94226882.2,1994-03-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,patent202210,马鞍山钢铁股份有限公司
3,131,具有抗静电性的轻垢液体洗衣洗涤剂,Liquid detergent with anti-static light scale ...,本发明提供一种透明或半透明的具有抗静电性 的轻垢液体洗衣洗涤剂，包括(以重量计)5—20％合...,The patent refers to the field of 'detergent c...,山东金泰集团股份有限公司,CN1094440A,1994-11-02,CN94110539.3,1994-04-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,patent202210,山东金泰集团股份有限公司
4,140,混铁炉柔性燃气供气装置,Flexible gas supply device for blast furnace m...,混铁炉柔性燃气供气装置属于炼钢技术领域。 它解决了原混铁炉燃气供气装置中的球形回转式弯 管部...,The utility model relates to a flexible gas su...,马鞍山钢铁股份有限公司,CN2192627Y,1995-03-22,CN94210226.6,1994-04-29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,patent202210,马鞍山钢铁股份有限公司


In [12]:
patent_all_invest.columns

Index(['序号', '标题 (中文)', '标题 (英文)', '摘要 (中文)', '摘要 (英文)', '申请人', '公开（公告）号',
       '公开（公告）日', '申请号', '申请日',
       ...
       'DWPI同族专利', 'DWPI同族国家/地区', 'DWPI同族ID', 'DWPI专利权人/申请人', 'DWPI发明人',
       'DWPI优先权国别-最早', 'DWPI优先权号', 'DWPI优先权国别', 'table_name', 'invest_申请人'],
      dtype='object', length=216)

In [15]:
patent_all_invest=pd.read_csv('data/patent_all_invest_2024.tsv',sep='\t',usecols=['申请人', '公开（公告）号',
       '公开（公告）日', '申请号', '申请日','table_name','invest_申请人'])
patent_all_invest

,申请人,公开（公告）号,公开（公告）日,申请号,申请日,table_name,invest_申请人
0,华东冶金学院; 马鞍山钢铁股份有限公司,CN2170307Y,1994-06-29,CN93237739.4,1993-10-11,patent202210,马鞍山钢铁股份有限公司
1,马鞍山钢铁股份有限公司,CN2204409Y,1995-08-02,CN94226705.2,1994-01-27,patent202210,马鞍山钢铁股份有限公司
2,马鞍山钢铁股份有限公司,CN2189729Y,1995-02-15,CN94226882.2,1994-03-04,patent202210,马鞍山钢铁股份有限公司
3,山东金泰集团股份有限公司,CN1094440A,1994-11-02,CN94110539.3,1994-04-14,patent202210,山东金泰集团股份有限公司
4,马鞍山钢铁股份有限公司,CN2192627Y,1995-03-22,CN94210226.6,1994-04-29,patent202210,马鞍山钢铁股份有限公司
...,...,...,...,...,...,...,...
2447662,深圳市智绘科技有限公司,CN308444862S,2024-01-30,CN202330846916.9,2023-12-22,patent_incopat_202212_202312,深圳市智绘科技有限公司
2447663,康力电梯股份有限公司,CN308439638S,2024-01-26,CN202330849053.0,2023-12-25,patent_incopat_202212_202312,康力电梯股份有限公司
2447664,广东新宝电器股份有限公司,CN308428257S,2024-01-19,CN202330862430.4,2023-12-28,patent_incopat_202212_202312,广东新宝电器股份有限公司
2447665,广东新宝电器股份有限公司,CN308428258S,2024-01-19,CN202330862435.7,2023-12-28,patent_incopat_202212_202312,广东新宝电器股份有限公司


In [16]:
patent_all_invest.table_name.value_counts()

table_name
patent202210                    826398
patent202404                    748835
patent_incopat_202101_202211    673656
patent_incopat_202212_202312     68403
patent202410                     56029
patent202309                     33314
patent202407                     31495
patent202408                      9537
Name: count, dtype: int64

patent_incopat_2016b_202411     6688311  
patent_incopat_202101_202211    5586621  
patent_incopat_2020b_202411     5464605  
patent202210                    1576086   
patent202404                    1492032  
patent202410                     688260  
patent_incopat_202212_202312     586203  
patent202309                      93815  
patent202408                      46033  
patent202407                      33097  

patent_incopat_2016b_202411,patent_incopat_2020b_202411

'patent_data_202411.tsv''patent_data_202411_2.tsv'  
china vc  
高新专利转让  

In [14]:
import os,glob
from pathlib import Path
path1=u'H:/Source/Patent/INCOPAT中国发明申请数据/2021-2022'
#os.chdir(path1)
file_path=os.listdir(path1)
file_path

['2022-10-16_1.xls',
 '2022-10-16_2.xls',
 '2022-10-16_3.xls',
 '2022-10-16_4.xls',
 '2022-10-16_5.xls',
 '2022-10-16_6.xls',
 '2022-10-16_7.xls',
 '2022-10-16_8.xls',
 '2022-10-18_1.xls',
 '2022-10-18_2.xls',
 '2022-10-18_3.xls',
 '2022-10-18_4.xls',
 '2022-10-18_5.xls',
 '2022-10-19_1.xls',
 '2022-10-19_2.xls',
 '2022-10-19_3.xls',
 '2022-10-19_4.xls',
 '2022-10-19_5.xls',
 '2022-10-19_6.xls',
 '2022-10-19_7.xls',
 '2022-10-21_1.xls',
 '2022-10-21_2.xls',
 '2022-10-21_3.xls',
 '2022-10-21_4.xls',
 '2022-10-21_5.xls',
 '2022-10-21_6.xls',
 '2022-10-22-2.xls',
 '2022-10-22-3.xls',
 '2022-10-22-4.xls',
 '2022-10-22-5.xls',
 '2022-10-22-6.xls',
 '2022-10-22.xls',
 '2022-10-23-2.xls',
 '2022-10-23.xls',
 '2022-10-24_2.xls',
 '2022-10-24_3.xls',
 '2022-10-25_1.xls',
 '2022-10-25_2.xls',
 '2022-10-25_3.xls']

In [35]:
len(file_path)

39

In [20]:
path2=u'H:/Source/Patent/INCOPAT中国发明申请数据/2023'
#os.chdir(path2)
file_path2=os.listdir(path2)
file_path2

['2024-03-11 (1).xlsx',
 '2024-03-11 (10).xlsx',
 '2024-03-11 (11).xlsx',
 '2024-03-11 (12).xlsx',
 '2024-03-11 (13).xlsx',
 '2024-03-11 (14).xlsx',
 '2024-03-11 (15).xlsx',
 '2024-03-11 (16).xlsx',
 '2024-03-11 (17).xlsx',
 '2024-03-11 (18).xlsx',
 '2024-03-11 (19).xlsx',
 '2024-03-11 (2).xlsx',
 '2024-03-11 (3).xlsx',
 '2024-03-11 (4).xlsx',
 '2024-03-11 (5).xlsx',
 '2024-03-11 (6).xlsx',
 '2024-03-11 (7).xlsx',
 '2024-03-11 (8).xlsx',
 '2024-03-11 (9).xlsx',
 '2024-03-11.xlsx',
 '2024-03-12 (1).xlsx',
 '2024-03-12 (2).xlsx',
 '2024-03-12 (3).xlsx',
 '2024-03-12 (4).xlsx',
 '2024-03-12 (5).xlsx',
 '2024-03-12 (6).xlsx',
 '2024-03-12 (7).xlsx',
 '2024-03-12 (8).xlsx',
 '2024-03-12 (9).xlsx',
 '2024-03-12.xlsx',
 '2024-03-15 (1).xlsx',
 '2024-03-15 (2).xlsx',
 '2024-03-15 (3).xlsx',
 '2024-03-15 (4).xlsx',
 '2024-03-15 (5).xlsx',
 '2024-03-15 (6).xlsx',
 '2024-03-15 (7).xlsx',
 '2024-03-15 (8).xlsx',
 '2024-03-15 (9).xlsx',
 '2024-03-15.xlsx',
 '2024-03-18 (1).xlsx',
 '2024-03-18 (2).x

In [34]:
len(file_path2)

230

In [24]:
os.chdir('E:\Documents\Jupyter')

In [15]:
path1 = r'H:/Source/Patent/INCOPAT中国发明申请数据/2021-2022'
path2 = r'H:/Source/Patent/INCOPAT中国发明申请数据/2023'

# 使用递归通配符 ** 匹配所有子目录
matching_files = []
for base_path in [path1, path2]:
    pattern = os.path.join(base_path, "**", "*.xls*")
    matching_files += glob.glob(pattern, recursive=True)

print(f"共找到 {len(matching_files)} 个 .xlsx 文件")

共找到 269 个 .xlsx 文件


In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [ ]:
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

In [18]:
data_columns=pd.read_excel(matching_files[-1],nrows=10).columns.tolist()
matching_files[-1],len(data_columns)

('H:/Source/Patent/INCOPAT中国发明申请数据/2023\\2024-04-03.xlsx', 210)

In [31]:
os.path.basename(matching_files[-1])

'2024-04-03.xlsx'

In [20]:
%%time
sizes=0
output_path = 'E:/Documents/Jupyter/data/source/patent_data_202507_3.tsv'#202411_3
patent_filter = pd.DataFrame()
for i,f_patent in enumerate(matching_files):
    df_chunk=pd.read_excel(f_patent)#,dtype=data_types, error_bad_lines=True
    
    print(i,sizes,f_patent,df_chunk.shape,df_chunk['序号'].max())#,patent_df.shape

    df_chunk['申请日']=pd.to_datetime(df_chunk['申请日'])
    df_chunk['公开（公告）日']=pd.to_datetime(df_chunk['公开（公告）日'])
    df_chunk.sort_values(['申请日','公开（公告）日'],inplace=True)
    df_chunk.reset_index(drop=True,inplace=True)
    
    df_chunk['序号'] = df_chunk.index + sizes + 1

    # try:
    #     df_chunk.to_sql('patent_incopat_2016b_202411', con=engine,chunksize=10000, if_exists='append', index=False)#,dtype=dtypes
    # except Exception as e:
    #     print(f"Error writing to SQL: {e}")

    sizes+=df_chunk.shape[0]
    
    #applicants=df_chunk['申请人'].str.split(';').explode().str.strip()
    #holders=df_chunk['当前权利人'].str.split(';').explode().str.strip()
    #filter_applicants=applicants[applicants.isin(gx_companies)]
    df_chunk['path'] = os.path.basename(f_patent)

    # if i<=44:
    #     continue
    with open(output_path,'a',encoding="utf_8_sig") as f: 
        # df_chunk.rename(
        #      columns= {'提出实审时长':'进入实质审查时长',
        #     '申请人国别代码':'申请人国家/地区',
        #     '发明人国别':'发明人国家/地区',
        #     '发明人地址':'发明人地址（原始）',}, inplace=True)
        df_chunk.reindex(columns=data_columns+['path'], fill_value=None).to_csv(f,sep='\t', mode="a", index=False,header=f.tell()==0)#使用空值填充缺失的数据columns=data_columns,na_rep='NaN',转csv合并
        
    del df_chunk

0 0 H:/Source/Patent/INCOPAT中国发明申请数据/2021-2022\2022-10-16_1.xls (60000, 175) 60000
1 60000 H:/Source/Patent/INCOPAT中国发明申请数据/2021-2022\2022-10-16_2.xls (60000, 175) 120000
2 120000 H:/Source/Patent/INCOPAT中国发明申请数据/2021-2022\2022-10-16_3.xls (60000, 175) 180000
3 180000 H:/Source/Patent/INCOPAT中国发明申请数据/2021-2022\2022-10-16_4.xls (38846, 175) 218846
4 218846 H:/Source/Patent/INCOPAT中国发明申请数据/2021-2022\2022-10-16_5.xls (25477, 175) 25477
5 244323 H:/Source/Patent/INCOPAT中国发明申请数据/2021-2022\2022-10-16_6.xls (10847, 175) 10847
6 255170 H:/Source/Patent/INCOPAT中国发明申请数据/2021-2022\2022-10-16_7.xls (26141, 175) 26141
7 281311 H:/Source/Patent/INCOPAT中国发明申请数据/2021-2022\2022-10-16_8.xls (8174, 175) 8174
8 289485 H:/Source/Patent/INCOPAT中国发明申请数据/2021-2022\2022-10-18_1.xls (57817, 175) 297817
9 347302 H:/Source/Patent/INCOPAT中国发明申请数据/2021-2022\2022-10-18_2.xls (60000, 175) 240000
10 407302 H:/Source/Patent/INCOPAT中国发明申请数据/2021-2022\2022-10-18_3.xls (60000, 175) 180000
11 467302 H:/Source/Patent/INCOPA

合并一下这部分的数据，我记得之前好像合并过？然后构建一下专利的每年的引用对的数据，主要用到的是下面的变量 df[['序号', '标题 (中文)', '标题 (英文)', '摘要 (中文)', '摘要 (英文)', '申请人', '公开（公告）号','引证申请人','申请日','自引信息','引证次数','公开（公告）日','引证专利']]，需要把每一个申请号对应的引证专利拆分之后，形成申请号-引证专利-申请日这样的结构，然后把引证申请人和引证专利同步的对应一下，之后再删去所有没有发生过引用的专利，并检索匹配一下引证专利对应的ipc分类号、摘要和标题以及法律状态这类的信息～

税调公司共同申请
税调与专利

In [21]:
use_cols=['序号', '标题 (中文)', '标题 (英文)', '摘要 (中文)', '摘要 (英文)', '申请人', '申请号','公开（公告）号','引证申请人','申请日','自引信息','引证次数','公开（公告）日','引证专利','IPC主分类', '法律状态']

In [22]:
%%time
patent_df_3 =pd.read_csv('data/source/patent_data_202507_3.tsv',usecols=use_cols,sep='\t')
patent_df_3.shape#3841167

<timed exec>:1: DtypeWarning: Columns (86) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 14min 57s
Wall time: 15min 47s


(3841167, 16)

In [24]:
patent_df_3.tail()

,序号,标题 (中文),标题 (英文),摘要 (中文),摘要 (英文),申请人,公开（公告）号,公开（公告）日,申请号,申请日,IPC主分类,引证专利,自引信息,引证申请人,引证次数,法律状态
3841162,3841163,一种注射笔,Injection pen,本申请公开了一种注射笔，包括按钮机构和剂量调节机构，所述按钮机构包括按钮组件，所述按钮组件包...,The present application discloses an injection...,苏州嘉树医疗科技有限公司,CN117563086B,2024-03-15,CN202410058059.X,2024-01-16,A61M5/315,WO2022217875A1; US20230398303A1; CN219814897U;...,CN219835963U,深圳惠迈智能科技有限公司; 赛诺菲阿凡提斯德意志有限公司; Adient Engineeri...,10.0,#法律状态公告日：20240220;\r\n法律状态：公开;\r\n描述信息：公开;\r\n...
3841163,3841164,一种注射笔及其剂量调节机构,The present invention relates to an injection ...,本申请公开了一种注射笔及其剂量调节机构，该剂量调节机构包括旋钮、止位支架和扭簧支架；旋钮能够...,The present application discloses an injection...,苏州嘉树医疗科技有限公司,CN117563085B,2024-03-15,CN202410058049.6,2024-01-16,A61M5/315,CN106267475A; CN219835963U; CN116271336A; CN21...,CN219835963U; CN116271336A,江苏德尔福医疗器械有限公司; 苏州嘉树医疗科技有限公司; 诺和诺德股份有限公司; Novo ...,6.0,#法律状态公告日：20240220;\r\n法律状态：公开;\r\n描述信息：公开;\r\n...
3841164,3841165,一种注射笔及其剂量调节机构,The present invention relates to an injection ...,本申请公开了一种注射笔及其剂量调节机构，该剂量调节机构包括动力杆、筒体、滚珠、套筒和扭簧；所...,The present application discloses an injection...,苏州嘉树医疗科技有限公司,CN117563084B,2024-03-15,CN202410058074.4,2024-01-16,A61M5/20,CN116650763A; CN113499513A; CN204133090U; CN21...,CN116271336A,诺和诺德股份有限公司; 苏州赛卫生物科技有限公司; 李民; 山东威高普瑞医药包装有限公司; ...,11.0,#法律状态公告日：20240220;\r\n法律状态：公开;\r\n描述信息：公开;\r\n...
3841165,3841166,一种碘化多羟基聚合物、制备方法及其制备的液体栓塞剂,"Iodinated polyhydroxy polymer, preparation me...",本发明公开了一种碘化多羟基聚合物、制备方法及其制备的液体栓塞剂，所述碘化多羟基聚合物的制备方...,The present invention discloses an iodinated p...,苏州美创医疗科技有限公司; 苏州美脉医疗器械有限公司,CN117586440A,2024-02-23,CN202410065737.5,2024-01-17,C08F16/06,US20130108574A1; US20210015963A1; CN114805644A...,CN114805644A,Biocompatibles Uk Limited; 天津南开和成科技有限公司; Yves ...,6.0,#法律状态公告日：20240223;\r\n法律状态：公开;\r\n描述信息：公开;\r\n...
3841166,3841167,睡眠领域技术研究转化的预警方法、装置、电子设备及存储介质,NaN,本申请提供了睡眠领域技术研究转化的预警方法、装置、电子设备及存储介质，确定睡眠领域的目标科学...,NaN,北京大众益康科技有限公司,CN117808633A,2024-04-02,CN202410223905.9,2024-02-29,G06Q50/18,NaN,NaN,NaN,NaN,#法律状态公告日：20240402;\r\n法律状态：公开;\r\n描述信息：公开;\r\n...


In [10]:
dx=pd.read_excel('H:/Source/Patent/INCOPAT中国发明申请数据/2023\\2024-04-03.xlsx',usecols=use_cols)

D:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [12]:
dx.iloc[-1]

序号                                                     13721
标题 (中文)                                              照相机用三角架
标题 (英文)                    Tripods for cameras [photography]
摘要 (中文)                                                  NaN
摘要 (英文)                                                  NaN
申请人                           Guangdong Sirui Optical Co Ltd
公开（公告）号                                     EU0063951820001S
公开（公告）日                                  2019-05-13 00:00:00
申请号                                              EU006395182
申请日                                      2019-04-24 00:00:00
IPC主分类                                                   NaN
引证专利                                                     NaN
自引信息                                                     NaN
引证申请人                                                    NaN
引证次数                                                     NaN
法律状态       #法律状态公告日：20190424;\n状态代码：RES;\n法律状态：;\n描述信息：;\...
Name: 3720, dtype: objec

In [25]:
del dx

In [26]:
import gc
gc.collect()

10

In [31]:
%%time
patent_df =pd.concat([pd.read_csv('data/source/patent_data_202411.tsv',sep='\t',usecols=use_cols),
                      pd.read_csv('data/source/patent_data_202411_2.tsv',sep='\t',usecols=use_cols),
                      patent_df_3], ignore_index=True) #chunksize=100000,,dtype=data_types,parse_dates=parse_dates
patent_df.shape

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [32]:
del patent_df_3
gc.collect()

1105

In [ ]:
#patent_df.to_csv('data/source/patent_all_invent_2507.tsv',sep='\t', index=False)

In [33]:
%%time
output_csv='data/source/patent_all_invent_2507.tsv'
if os.path.exists(output_csv):
    os.remove(output_csv)
for p in ['data/source/patent_data_202411.tsv','data/source/patent_data_202411_2.tsv','data/source/patent_data_202507_3.tsv']:
    for i,df_chunk in enumerate(pd.read_csv(p,sep='\t',usecols=use_cols,chunksize=100000)):
        df_chunk['source']=p
        df_chunk.reindex(columns=use_cols+['source'], fill_value=None).to_csv(output_csv,sep='\t', mode="a", index=False,header=not os.path.exists(output_csv) and i == 0)

<timed exec>:5: DtypeWarning: Columns (86) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 2h 50min 22s
Wall time: 2h 50min 57s


In [34]:
i

38

In [3]:
output_csv='data/source/patent_all_invent_2507.tsv'
use_cols_2=['序号', '申请人', '申请号','申请日','公开（公告）号','公开（公告）日','引证申请人','自引信息','引证次数','引证专利','IPC主分类']

In [6]:
%%time
patent_df=pd.read_csv(output_csv,sep='\t',usecols=use_cols_2+['source'])
patent_df.shape

<timed exec>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 8min 8s
Wall time: 8min 18s


(17075032, 12)

In [38]:
patent_df['公开（公告）号'].nunique()

15522559

In [39]:
patent_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17075032 entries, 0 to 17075031
Data columns (total 11 columns):
 #   Column   Dtype  
---  ------   -----  
 0   序号       int64  
 1   申请人      object 
 2   申请号      object 
 3   公开（公告）号  object 
 4   引证申请人    object 
 5   申请日      object 
 6   自引信息     object 
 7   引证次数     float64
 8   公开（公告）日  object 
 9   引证专利     object 
 10  IPC主分类   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 1.4+ GB


In [2]:
from dask.distributed import Client
import dask.dataframe as dd
client = Client("tcp://10.10.10.20:8786")
client

D:\Program Files\Python\Python313\Lib\site-packages\distributed\client.py:1582: VersionMismatchWarning: Mismatched versions found

+-------------+----------------+----------------+--------------------------------------------------------+
| Package     | Client         | Scheduler      | Workers                                                |
+-------------+----------------+----------------+--------------------------------------------------------+
| cloudpickle | 3.1.1          | 3.1.1          | {'3.0.0', '3.1.1'}                                     |
| dask        | 2025.5.1       | 2025.5.1       | {'2023.12.1+dfsg', '2024.7.1', '2025.5.1'}             |
| distributed | 2025.5.1       | 2025.5.1       | {'2025.5.1', '2024.7.1', '0+unknown'}                  |
| lz4         | None           | None           | {None, '4.0.2+dfsg'}                                   |
| msgpack     | 1.1.0          | 1.1.1          | {'1.0.3', '1.0.8', '1.1.0'}                            |
| numpy      

<Client: 'tcp://10.10.10.20:8786' processes=3 threads=36, memory=87.01 GiB>

2025-07-16 00:30:57,292 - distributed.batched - INFO - Batched Comm Closed <TCP (closed) Client->Scheduler local=tcp://172.16.0.213:49948 remote=tcp://10.10.10.20:8786>
Traceback (most recent call last):
  File "D:\Program Files\Python\Python313\Lib\site-packages\distributed\batched.py", line 115, in _background_send
    nbytes = yield coro
             ^^^^^^^^^^
  File "D:\Program Files\Python\Python313\Lib\site-packages\tornado\gen.py", line 766, in run
    value = future.result()
  File "D:\Program Files\Python\Python313\Lib\site-packages\distributed\comm\tcp.py", line 263, in write
    raise CommClosedError()
distributed.comm.core.CommClosedError
2025-07-16 02:30:57,640 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [7]:
# import dask
# df=dask.datasets.timeseries()
# df.head()
patent_df =dd.from_pandas(df_pd, npartitions=4)

In [17]:
import csv

In [4]:
%%time
patent_df= dd.read_csv(output_csv,sep='\t',usecols=use_cols_2,  blocksize=None)#, on_bad_lines='skip', encoding='utf-8', quoting=csv.QUOTE_MINIMAL
patent_df.shape

CPU times: total: 15.6 ms
Wall time: 84.2 ms


(<dask_expr.expr.Scalar: expr=ArrowStringConversion(frame=FromMapProjectable(dcc2b2b)).size() // 11, dtype=int64>,
 11)

In [5]:
len(patent_df) 

MemoryError: 

In [10]:
patent_df['公开（公告）日']=pd.to_datetime(patent_df['公开（公告）日'])
patent_df['申请日']=pd.to_datetime(patent_df['申请日'])
patent_df=patent_df.sort_values(['申请日','公开（公告）日']).drop_duplicates(['申请号','公开（公告）号'],keep='first').reset_index()#drop=True
patent_df.shape,patent_df['申请号'].nunique()

((15522559, 13), 14961771)

In [41]:
patent_df['公开（公告）日'].max(),patent_df['申请日'].max()

(Timestamp('2024-11-19 00:00:00'), Timestamp('2024-03-07 00:00:00'))

In [44]:
patent_df[['引证专利','引证申请人']].dropna(how='all')

,引证专利,引证申请人
2,US3979801A; US3765062A; US302421A; US796414A; ...,Walker Ltd Thomas; Tareau Georges; Bonnie Ente...
3,US4166180A,Hokuriku Pharmaceutical
5,DE1302592B; DE1302592C2,NaN
7,FR2499712B1; FR2499712A1,Etat Francais; France Etat
9,DE2747765B2; DE2747765A1,"Walter Rau Lebensmittelwerke, 4517 Hilter; Ra..."
...,...,...
15522545,CN103811463A; CN111769097A; CN107611081A; US20...,蔡佳勋; 陈鼎钧; 杨宏智; 唐肇蔚; 黄希哲; 廖学专; David Edward Fis...
15522546,CN102957303A; US5528125A; CN105024542A; CN1011...,苏州贝克微电子有限公司; Texas Instruments Inc; 张云山; 林大伟; ...
15522547,CN209545214U; US20100019807A1; CN101291103A; C...,苏州威尔阳光智能科技有限公司; 普联技术有限公司; 思柏科技股份有限公司; 南京微盟电子有限...
15522548,CN101093843A; CN101303500A; CN111198463A; US20...,深圳市华星光电半导体显示技术有限公司; Chikaaki Mizoguchi; Samsun...


In [11]:
patent_df.to_parquet('data/source/patent_all_invent_25070.parquet',index=False)
patent_df.columns#'source''index'

Index(['index', '序号', '申请人', '申请号', '公开（公告）号', '引证申请人', '申请日', '自引信息', '引证次数',
       '公开（公告）日', '引证专利', 'IPC主分类', 'source'],
      dtype='object')

In [12]:
patent_df['引证专利']=patent_df['引证专利'].fillna('').str.split(';')#拆分引用边
#patent_df['引证申请人']=patent_df['引证申请人'].fillna('').str.split(';')
exploded = patent_df[['申请号','申请日', '申请人','公开（公告）号', '引证专利', '引证申请人']].explode('引证专利')
exploded['引证专利'] = exploded['引证专利'].str.strip()
exploded = exploded[exploded['引证专利'] != '']
len(exploded)#70714339

70714339

In [50]:
exploded.引证专利.isna().sum()

0

In [26]:
exploded['引证专利'].value_counts()

引证专利
US4946778A      6642
US4816567A      5872
US4683202A      3549
US4683202B1     3535
US5585089A      2942
                ... 
CN203284914U       1
CN205224359U       1
CN205498192U       1
CN201024380Y       1
TW201944584A       1
Name: count, Length: 25827209, dtype: int64

In [13]:
all_nodes = sorted(set(exploded['公开（公告）号'].dropna().unique()) | set(exploded['引证专利'].dropna().unique()))
len(all_nodes)

30370818

In [15]:
idx_map = {node: i for i, node in enumerate(all_nodes)}
sources = exploded['公开（公告）号'].map(idx_map).tolist()
targets = exploded['引证专利'].map(idx_map).tolist()
all_edges =list(zip(sources, targets))

In [17]:
del sources
del targets
len(all_edges)#70714339

70714339

In [22]:
edge_weights = Counter(all_edges)
len(edge_weights)#70714339,同个专利重复引证？

NameError: name 'Counter' is not defined

In [ ]:
edges, weights = zip(*edge_weights.items())

In [18]:
import gc
gc.collect()

565

In [19]:
import igraph as ig
from igraph import plot

In [20]:
g = ig.Graph(directed=True)
g.add_vertices(len(all_nodes))
g.vs['name'] = all_nodes
g.add_edges(all_edges) #edges citing
# g.es['weight'] = weights

In [23]:
g.vs.attributes(), g.es.attributes()

(['name'], [])

In [24]:
g.write_gml("data//patent_发明引证_graph_adjlist_2024.gml")#write_adjlist

In [21]:
g.write_pickle("data//patent_发明引证_graph_2024.pickle")#write_gml,write_graphml...

In [25]:
del all_nodes,all_edges

In [ ]:
df_nodes = pd.DataFrame({'name': all_nodes}).merge(patent_df[['公开（公告）号','申请号', 'IPC主分类', '标题 (中文)', '摘要 (中文)', '法律状态', '申请人']].drop_duplicates(subset=['公开（公告）号']).rename(columns={'公开（公告）号': 'name'}), on='name', how='left')
for attr in ['IPC主分类', '标题 (中文)', '摘要 (中文)', '法律状态', '申请人']:
    g.vs[attr] = df_nodes[attr].fillna('').tolist()

In [ ]:
records = []
for e in g.es:
    src, dst = e.tuple
    records.append({
        'source': g.vs[src]['name'],
        'target': g.vs[dst]['name'],
        'IPC主分类': g.vs[dst]['IPC主分类'],
        '摘要 (中文)': g.vs[dst]['摘要 (中文)'],
        '标题 (中文)': g.vs[dst]['标题 (中文)'],
        '法律状态': g.vs[dst]['法律状态']
    })
return pd.DataFrame(records)

In [ ]:
node_df = pd.DataFrame({attr: g.vs[attr] for attr in g.vs.attributes()})

In [ ]:
edge_df = pd.DataFrame({
    'source': [g.vs[e.source]['name'] for e in g.es],
    'target': [g.vs[e.target]['name'] for e in g.es]
})
for attr in g.es.attributes():
    edge_df[attr] = g.es[attr]

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']='SimHei'